<a href="https://colab.research.google.com/github/GriPet12/memcoin-graduation/blob/main/memcoin_graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [6]:
!pip install catboost lightgbm xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 3.3 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
import glob
import os
import gc
from tqdm.auto import tqdm

# Modeling & Evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.metrics import log_loss

# Configuration

In [8]:
DATA_PATH = '/content/drive/MyDrive/memcoin-graduation/pump-fun-graduation-february-2025'
CHUNK_PATTERN = os.path.join(DATA_PATH, 'chunk*.csv')
TRAIN_FILE = os.path.join(DATA_PATH, 'train.csv')
TEST_FILE = os.path.join(DATA_PATH, 'test_unlabeled.csv')
DUNE_INFO_FILE = os.path.join(DATA_PATH, 'dune_token_info.csv')
ONCHAIN_INFO_FILE = os.path.join(DATA_PATH, 'token_info_onchain_divers.csv')
SUBMISSION_FILE = 'submission.csv'

TARGET = 'has_graduated'
MINT_ID = 'mint'
BLOCK_LIMIT = 100 # Only use data from first 100 blocks post-mint
N_SPLITS = 5 # Number of folds for cross-validation
RANDOM_SEED = 42

# Load Data

In [9]:
print("Loading data...")
train_df = pd.read_csv(TRAIN_FILE)
test_df = pd.read_csv(TEST_FILE)
dune_info_df = pd.read_csv(DUNE_INFO_FILE)
onchain_info_df = pd.read_csv(ONCHAIN_INFO_FILE)

train_df['is_train'] = 1
test_df['is_train'] = 0
combined_df = pd.concat([train_df, test_df], ignore_index=True)

all_chunk_files = glob.glob(CHUNK_PATTERN)
print(f"Found {len(all_chunk_files)} chunk files.")

chunk_list = []
for f in tqdm(all_chunk_files, desc="Loading chunks"):
    try:
        chunk_list.append(pd.read_csv(f))
    except Exception as e:
        print(f"Error loading {f}: {e}")
if not chunk_list:
    raise ValueError("No chunk files loaded. Check CHUNK_PATTERN and file existence.")

transactions_df = pd.concat(chunk_list, ignore_index=True)

Loading data...


<ipython-input-9-3ee4f091b291>:5: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  onchain_info_df = pd.read_csv(ONCHAIN_INFO_FILE)


Found 41 chunk files.


Loading chunks:   0%|          | 0/41 [00:00<?, ?it/s]

In [10]:
transactions_df.head()

,block_time,slot,tx_idx,signing_wallet,direction,base_coin,base_coin_amount,quote_coin_amount,virtual_token_balance_after,virtual_sol_balance_after,signature,provided_gas_fee,provided_gas_limit,fee,consumed_gas
0,2025-02-01 16:00:00,317876496,1897,EXoaGQc1taATjsXVPXhnVYddW4KiM1uQRhrYDdfi1x7b,buy,Ab2voNJxp9xM2sdoF6JRJV8dtZ6hGm8yMSt3xAMpump,5208861189189,150740503,1052069532604495,30596837025,3Td5mZpy63TNuyHncgpJFvPudbU3fKcRaeTbqTRsdSLKnN...,3982833,200862,805000,161488
1,2025-02-01 16:00:00,317876496,1794,9Ypu1cMva6dE6k9Zk4aSSmSgJvMmJLTWXuGQhTYqt8mx,buy,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,1785357737104,50000000,1071214642262896,30050000000,3HHvJsNKWg6epToaZUouqDJdkysiJGKeBkWnhg7sPyWpFY...,9475209,194994,1857609,194994
2,2025-02-01 16:00:00,317876496,1880,3njxeVx5TjDYD27C1YsZW2JQzgmoYeATambcbw7Xn1ft,sell,FZ8wX1RAwV72gniwc9quiZSXHnrECQwoCxAXCWRipump,721068391933,22610289,1013561711601751,31759289915,2rfHemwRWv9t2xuY2umAq2aQrx8Gn73g6tAEDBTE2yrzcf...,4624039,162000,754095,78540
3,2025-02-01 16:00:01,317876499,1350,DjZ1Cpxp6uKvYHU678QkjFj8XKfUwAmCtxUMG5QuBdJT,buy,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,115855160976852,3644123135,955359481286044,33694123135,Ky5DURUWgB7N3NfNAx5jf7a23pecQAwxbcwuwaWE2MPk7R...,170888000,80000,13676040,62135
4,2025-02-01 16:00:01,317876500,2389,6WgXuHPo9xWu1Mzt8hULYWFwpyh1WwhByPYjsF23h41A,buy,4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump,67062499999999,2000000000,1005937500000001,32000000000,4rSq4SK7a6zb2hNV6x89DfwZ9XFAyy4sp1g9YQfuG8A3Ep...,100000,500000,60000,201798


In [11]:
transactions_df['block_time'] = pd.to_datetime(transactions_df['block_time'], errors='coerce')
transactions_df['slot'] = pd.to_numeric(transactions_df['slot'], errors='coerce')
combined_df['slot_min'] = pd.to_numeric(combined_df['slot_min'], errors='coerce')

# Data Merging and Preprocessing

In [12]:
print("Merging data...")

transactions_df = pd.merge(
    transactions_df,
    combined_df[[MINT_ID, 'slot_min']],
    left_on='base_coin',
    right_on=MINT_ID,
    how='left'
)

transactions_df = transactions_df[
    transactions_df['slot'] <= transactions_df['slot_min'] + BLOCK_LIMIT
    ]
transactions_df.columns

Merging data...


Index(['block_time', 'slot', 'tx_idx', 'signing_wallet', 'direction',
       'base_coin', 'base_coin_amount', 'quote_coin_amount',
       'virtual_token_balance_after', 'virtual_sol_balance_after', 'signature',
       'provided_gas_fee', 'provided_gas_limit', 'fee', 'consumed_gas', 'mint',
       'slot_min'],
      dtype='object')

In [13]:
dune_info_df = dune_info_df.rename(columns={'token_mint_address': MINT_ID})
dune_info_df = dune_info_df[[MINT_ID, 'decimals', 'name', 'symbol', 'token_uri', 'created_at', 'init_tx']].drop_duplicates(subset=[MINT_ID], keep='first')
dune_info_df['created_at'] = pd.to_datetime(dune_info_df['created_at'], errors='coerce')

onchain_info_df = onchain_info_df.rename(columns={'mint': MINT_ID})
onchain_info_df = onchain_info_df[[MINT_ID, 'creator', 'bundle_size', 'gas_used']].drop_duplicates(subset=[MINT_ID], keep='first')
onchain_info_df['bundle_size'] = pd.to_numeric(onchain_info_df['bundle_size'], errors='coerce').fillna(0)
onchain_info_df['gas_used'] = pd.to_numeric(onchain_info_df['gas_used'], errors='coerce')

dune_info_df.columns, onchain_info_df.columns

(Index(['mint', 'decimals', 'name', 'symbol', 'token_uri', 'created_at',
        'init_tx'],
       dtype='object'),
 Index(['mint', 'creator', 'bundle_size', 'gas_used'], dtype='object'))

In [14]:
combined_df = pd.merge(combined_df, dune_info_df, on=MINT_ID, how='left')
combined_df = pd.merge(combined_df, onchain_info_df, on=MINT_ID, how='left')
combined_df.columns

Index(['Unnamed: 0', 'mint', 'slot_min', 'slot_graduated', 'has_graduated',
       'is_valid', 'is_train', 'decimals', 'name', 'symbol', 'token_uri',
       'created_at', 'init_tx', 'creator', 'bundle_size', 'gas_used'],
      dtype='object')

# Exploratory Data Analysis

In [16]:
print("Basic EDA (Conceptual):")
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Transactions shape (first 100 blocks): {transactions_df.shape}")
print(f"Combined shape before features: {combined_df.shape}")

print("\nMissing values in combined metadata:")
combined_df.isnull().sum() / len(combined_df)

Basic EDA (Conceptual):
Train shape: (639557, 7)
Test shape: (478832, 5)
Transactions shape (first 100 blocks): (16712389, 17)
Combined shape before features: (1118389, 16)

Missing values in combined metadata:


,0
Unnamed: 0,0.000000
mint,0.000000
slot_min,0.000000
slot_graduated,0.993380
has_graduated,0.428144
is_valid,0.000000
is_train,0.000000
decimals,0.208081
name,0.208302
symbol,0.208477


In [17]:
print("\nTarget Distribution:")
print(combined_df[TARGET].value_counts(normalize=True))


Target Distribution:
has_graduated
False    0.988423
True     0.011577
Name: proportion, dtype: float64


In [18]:
print("\nTransaction Data Info:")
transactions_df.info()


Transaction Data Info:
<class 'pandas.core.frame.DataFrame'>
Index: 16712389 entries, 1 to 17033441
Data columns (total 17 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   block_time                   datetime64[ns]
 1   slot                         int64         
 2   tx_idx                       int64         
 3   signing_wallet               object        
 4   direction                    object        
 5   base_coin                    object        
 6   base_coin_amount             int64         
 7   quote_coin_amount            int64         
 8   virtual_token_balance_after  int64         
 9   virtual_sol_balance_after    int64         
 10  signature                    object        
 11  provided_gas_fee             int64         
 12  provided_gas_limit           int64         
 13  fee                          int64         
 14  consumed_gas                 int64         
 15  mint                        

In [19]:
print("\nSample Transactions:")
transactions_df.head()


Sample Transactions:


,block_time,slot,tx_idx,signing_wallet,direction,base_coin,base_coin_amount,quote_coin_amount,virtual_token_balance_after,virtual_sol_balance_after,signature,provided_gas_fee,provided_gas_limit,fee,consumed_gas,mint,slot_min
1,2025-02-01 16:00:00,317876496,1794,9Ypu1cMva6dE6k9Zk4aSSmSgJvMmJLTWXuGQhTYqt8mx,buy,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,1785357737104,50000000,1071214642262896,30050000000,3HHvJsNKWg6epToaZUouqDJdkysiJGKeBkWnhg7sPyWpFY...,9475209,194994,1857609,194994,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,317876496.0
3,2025-02-01 16:00:01,317876499,1350,DjZ1Cpxp6uKvYHU678QkjFj8XKfUwAmCtxUMG5QuBdJT,buy,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,115855160976852,3644123135,955359481286044,33694123135,Ky5DURUWgB7N3NfNAx5jf7a23pecQAwxbcwuwaWE2MPk7R...,170888000,80000,13676040,62135,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,317876496.0
4,2025-02-01 16:00:01,317876500,2389,6WgXuHPo9xWu1Mzt8hULYWFwpyh1WwhByPYjsF23h41A,buy,4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump,67062499999999,2000000000,1005937500000001,32000000000,4rSq4SK7a6zb2hNV6x89DfwZ9XFAyy4sp1g9YQfuG8A3Ep...,100000,500000,60000,201798,4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump,317876500.0
5,2025-02-01 16:00:01,317876500,2398,3G3ARHTTF6tw5K3YzCciy9shCkz4SLfJgXn57UMUsAuP,buy,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,17590163934426,500000000,1055409836065574,30500000000,4Ms5ZBH3YFrzKHFjMauJaVnidxpdc1N1hDLBVeCUGy163o...,0,0,10000,187493,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,317876500.0
6,2025-02-01 16:00:01,317876500,2399,5ceEgLb1Kt3a6KdHVVA28EiNEiy3etiB2x75YEfH1VoC,buy,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,26975331273241,800000000,1028434504792333,31300000000,3ndYty7LJfNsML3n83DhXACNTwEahWeAdGBDXz7TABccKV...,0,0,5000,58911,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,317876500.0


# Feature Engineering

In [20]:
print("Starting Feature Engineering...")

grouped_tx = transactions_df.groupby('base_coin')

agg_funcs = {
    'tx_idx': ['count'],
    'block_time': ['min', 'max'],
    'slot': ['min', 'max', 'nunique'],
    'signing_wallet': ['nunique'],
    'quote_coin_amount': ['sum', 'mean', 'std', 'max'],
    'base_coin_amount': ['sum', 'mean', 'std', 'max'],
    'virtual_sol_balance_after': ['last', 'max', 'min', 'mean', 'std'],
    'virtual_token_balance_after': ['last', 'max', 'min', 'mean', 'std']
}

agg_features = grouped_tx.agg(agg_funcs)
agg_features.columns = ['_'.join(col).strip() for col in agg_features.columns.values]
agg_features = agg_features.reset_index().rename(columns={'base_coin': MINT_ID})

Starting Feature Engineering...


In [21]:
buy_tx = transactions_df[transactions_df['direction'] == 'buy']
sell_tx = transactions_df[transactions_df['direction'] == 'sell']

grouped_buy = buy_tx.groupby('base_coin')
grouped_sell = sell_tx.groupby('base_coin')

buy_agg = grouped_buy.agg({
    'tx_idx': ['count'],
    'signing_wallet': ['nunique'],
    'quote_coin_amount': ['sum', 'mean', 'max'],
    'base_coin_amount': ['sum', 'mean', 'max'],
}).reset_index()
buy_agg.columns = [MINT_ID] + ['buy_' + '_'.join(col).strip() for col in buy_agg.columns[1:]]

sell_agg = grouped_sell.agg({
    'tx_idx': ['count'],
    'signing_wallet': ['nunique'],
    'quote_coin_amount': ['sum', 'mean', 'max'],
    'base_coin_amount': ['sum', 'mean', 'max'],
}).reset_index()
sell_agg.columns = [MINT_ID] + ['sell_' + '_'.join(col).strip() for col in sell_agg.columns[1:]]

buy_agg.columns, sell_agg.columns,

(Index(['mint', 'buy_tx_idx_count', 'buy_signing_wallet_nunique',
        'buy_quote_coin_amount_sum', 'buy_quote_coin_amount_mean',
        'buy_quote_coin_amount_max', 'buy_base_coin_amount_sum',
        'buy_base_coin_amount_mean', 'buy_base_coin_amount_max'],
       dtype='object'),
 Index(['mint', 'sell_tx_idx_count', 'sell_signing_wallet_nunique',
        'sell_quote_coin_amount_sum', 'sell_quote_coin_amount_mean',
        'sell_quote_coin_amount_max', 'sell_base_coin_amount_sum',
        'sell_base_coin_amount_mean', 'sell_base_coin_amount_max'],
       dtype='object'))

In [22]:
print("Merging aggregated features...")

combined_df = pd.merge(combined_df, agg_features[[c for c in agg_features.columns if c != 'slot_min']], on=MINT_ID, how='left')

combined_df = pd.merge(combined_df, buy_agg, on=MINT_ID, how='left')

combined_df = pd.merge(combined_df, sell_agg, on=MINT_ID, how='left')

Merging aggregated features...


In [23]:
print("Calculating derived features...")

required_cols_for_duration = ['block_time_max', 'block_time_min', 'slot_max', 'slot_min', 'tx_idx_count', 'slot_nunique']

combined_df['tx_duration_seconds'] = (combined_df['block_time_max'] - combined_df['block_time_min']).dt.total_seconds()
combined_df['tx_duration_slots'] = combined_df['slot_max'] - combined_df['slot_min']
combined_df['avg_time_between_tx'] = combined_df['tx_duration_seconds'] / (combined_df['tx_idx_count'] + 1e-6)
combined_df['tx_per_slot'] = combined_df['tx_idx_count'] / (combined_df['slot_nunique'] + 1e-6)

Calculating derived features...


In [24]:
required_cols_for_ratios = [
    'buy_tx_idx_count', 'sell_tx_idx_count',
    'buy_quote_coin_amount_sum', 'sell_quote_coin_amount_sum',
    'buy_signing_wallet_nunique', 'sell_signing_wallet_nunique', 'signing_wallet_nunique'
]

combined_df['buy_sell_count_ratio'] = combined_df['buy_tx_idx_count'] / (combined_df['sell_tx_idx_count'] + 1e-6)
combined_df['buy_sell_vol_ratio'] = combined_df['buy_quote_coin_amount_sum'] / (combined_df['sell_quote_coin_amount_sum'] + 1e-6)
combined_df['unique_buyer_ratio'] = combined_df['buy_signing_wallet_nunique'] / (combined_df['signing_wallet_nunique'] + 1e-6)
combined_df['unique_seller_ratio'] = combined_df['sell_signing_wallet_nunique'] / (combined_df['signing_wallet_nunique'] + 1e-6)

In [25]:
creator_trades = transactions_df.groupby(['base_coin', 'signing_wallet']).size().reset_index(name='trade_count')

creator_trades = pd.merge(creator_trades, onchain_info_df[[MINT_ID, 'creator']], left_on='base_coin', right_on=MINT_ID, how='inner')
creator_trades = creator_trades[creator_trades['signing_wallet'] == creator_trades['creator']]
creator_trades = creator_trades[['base_coin', 'trade_count']].rename(columns={'base_coin': MINT_ID, 'trade_count': 'creator_trade_count'})
creator_trades = creator_trades.drop_duplicates(subset=[MINT_ID], keep='first')

In [26]:
combined_df = pd.merge(combined_df, creator_trades, on=MINT_ID, how='left')

combined_df['creator_traded'] = combined_df['creator_trade_count'].notna().astype(int)
combined_df['creator_trade_count'] = combined_df['creator_trade_count'].fillna(0)

# Final Feature Selection

In [27]:
print("Selecting final features...")
features_to_drop = [
    MINT_ID, TARGET, 'slot_graduated', 'is_train', 'slot_min',
    'name', 'symbol', 'token_uri', 'created_at', 'init_tx',
    'block_time_min', 'block_time_max',
    'creator',
    'is_valid', 'Unnamed: 0'
]

features = [col for col in combined_df.columns if col not in features_to_drop]
categorical_features = ['creator_encoded']

print(f"Using {len(features)} features: {features}")
for f in features:
    if combined_df[f].dtype == 'object':
        print(f"Warning: Feature '{f}' is object type. Ensure proper handling.")
        try:
            combined_df[f] = pd.to_numeric(combined_df[f])
        except:
            print(f"Could not convert {f} to numeric. Consider encoding or dropping.")
            if f in features: features.remove(f)


train_processed = combined_df[combined_df['is_train'] == 1].reset_index(drop=True)
train_processed[TARGET] = train_processed[TARGET].astype(int)
test_processed = combined_df[combined_df['is_train'] == 0].reset_index(drop=True)

X = train_processed[features]
y = train_processed[TARGET]
X_test = test_processed[features]

del combined_df, transactions_df, agg_features, buy_agg, sell_agg, creator_trades, chunk_list
gc.collect()

Selecting final features...
Using 51 features: ['decimals', 'bundle_size', 'gas_used', 'tx_idx_count', 'slot_max', 'slot_nunique', 'signing_wallet_nunique', 'quote_coin_amount_sum', 'quote_coin_amount_mean', 'quote_coin_amount_std', 'quote_coin_amount_max', 'base_coin_amount_sum', 'base_coin_amount_mean', 'base_coin_amount_std', 'base_coin_amount_max', 'virtual_sol_balance_after_last', 'virtual_sol_balance_after_max', 'virtual_sol_balance_after_min', 'virtual_sol_balance_after_mean', 'virtual_sol_balance_after_std', 'virtual_token_balance_after_last', 'virtual_token_balance_after_max', 'virtual_token_balance_after_min', 'virtual_token_balance_after_mean', 'virtual_token_balance_after_std', 'buy_tx_idx_count', 'buy_signing_wallet_nunique', 'buy_quote_coin_amount_sum', 'buy_quote_coin_amount_mean', 'buy_quote_coin_amount_max', 'buy_base_coin_amount_sum', 'buy_base_coin_amount_mean', 'buy_base_coin_amount_max', 'sell_tx_idx_count', 'sell_signing_wallet_nunique', 'sell_quote_coin_amount_su

8